In [5]:
!pip install ultralytics --upgrade -q
from ultralytics import YOLO
import os
!pip install ultralytics --quiet
import matplotlib.pyplot as plt
import cv2
import os
import pandas as pd
!pip install gdown

In [6]:
import gdown

# Replace this with your actual Google Drive sharable link
url = "https://drive.google.com/drive/folders/1CdlStMBYU_9KYEkU42P83LloQ6E2qqfh"
output = "best.pt"
gdown.download(url, output, quiet=False)

/opt/anaconda3/lib/python3.13/site-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://drive.google.com/drive/folders/1CdlStMBYU_9KYEkU42P83LloQ6E2qqfh
To: /Users/eraiyanbu/Downloads/best.pt
1.11MB [00:00, 6.72MB/s]


'best.pt'

In [9]:
DATASET_PATH = '/content/drive/MyDrive/Military/military_object_dataset'
%cd {DATASET_PATH}

/content/drive/MyDrive/Military/military_object_dataset


In [10]:
DATA_CONFIG_PATH = f'{DATASET_PATH}/military_datasetv1.yaml'

In [11]:
model = YOLO('yolov8n.yaml')  # You can switch to yolov8s.yaml or yolov8m.yaml for larger models

In [7]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import torch
print("CUDA Available:", torch.cuda.is_available())


CUDA Available: True


In [15]:
label_dirs = [
    '/content/drive/MyDrive/Military/military_object_dataset/train/labels',
    '/content/drive/MyDrive/Military/military_object_dataset/val/labels'
]

for label_dir in label_dirs:
    for file in os.listdir(label_dir):
        path = os.path.join(label_dir, file)
        with open(path, 'r') as f:
            lines = f.readlines()
        filtered = [line for line in lines if int(line.split()[0]) <= 3]
        with open(path, 'w') as f:
            f.writelines(filtered)

In [11]:
model = YOLO("yolov8n.pt")  # small model

model.train(
    data="/content/drive/MyDrive/Military/military_object_dataset/military_datasetv1.yaml",
    epochs=10,
    imgsz=640,
    batch=8,
    device=0,
    name="military_fast_4class"
)

100%|██████████| 6.25M/6.25M [00:00<00:00, 96.8MB/s]


Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Military/military_object_dataset/military_datasetv1.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=military_fast_4class, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, pa

100%|██████████| 755k/755k [00:00<00:00, 20.7MB/s]

Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Model summary: 129 layers, 3,011,628 parameters, 3,011,612 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


...

                   all       2941       2803      0.597      0.533      0.545      0.313



10 epochs completed in 1.087 hours.
Optimizer stripped from runs/detect/military_fast_4class/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/military_fast_4class/weights/best.pt, 6.2MB

Validating runs/detect/military_fast_4class/weights/best.pt...
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 184/184 [00:44<00:00,  4.16it/s]


                   all       2941       2803      0.597      0.533      0.545      0.313
               soldier        385        510      0.682      0.551      0.615      0.285
                   gun        222        358      0.516      0.399      0.394      0.255
                  tank        938       1787      0.735      0.775      0.783      0.498
                helmet         84        148      0.456      0.405      0.388      0.214
Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs/detect/military_fast_4class


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79b8abacfbd0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

In [12]:
# ✅ STEP 7: Evaluate the Model
metrics = model.val()
print(metrics)


Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.6±0.3 ms, read: 72.0±37.0 MB/s, size: 487.3 KB)


val: Scanning /content/drive/MyDrive/Military/military_object_dataset/val/labels.cache... 2941 images, 1322 backgrounds, 0 corrupt: 100%|██████████| 2941/2941 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 368/368 [01:18<00:00,  4.71it/s]


                   all       2941       2803      0.595      0.533      0.544      0.312
               soldier        385        510      0.682      0.551      0.615      0.285
                   gun        222        358      0.508      0.397      0.394      0.254
                  tank        938       1787      0.735      0.778      0.784      0.498
                helmet         84        148      0.456      0.405      0.383      0.213
Speed: 0.3ms preprocess, 4.7ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/detect/military_fast_4class2
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79b8ab591210>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,

In [17]:
model.export(format='torchscript')

Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/

PyTorch: starting from 'runs/detect/military_fast_4class/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 8, 8400) (5.9 MB)

TorchScript: starting export with torch 2.6.0+cu124...
TorchScript: export success ✅ 4.1s, saved as 'runs/detect/military_fast_4class/weights/best.torchscript' (11.9 MB)

Export complete (5.1s)
Results saved to /content/drive/MyDrive/Military/military_object_dataset/runs/detect/military_fast_4class/weights
Predict:         yolo predict task=detect model=runs/detect/military_fast_4class/weights/best.torchscript imgsz=640  
Validate:        yolo val task=detect model=runs/detect/military_fast_4class/weights/best.torchscript imgsz=640 data=/content/drive/MyDrive/Military/military_object_dataset/military_datasetv1.yam

'runs/detect/military_fast_4class/weights/best.torchscript'

In [19]:
# ✅ STEP 10: DONE
print("✅ Training and export complete.")

✅ Training and export complete.


In [28]:
# Load your trained model
model = YOLO('/content/drive/MyDrive/military_object_dataset/yolov8n.pt')  # adjust if saved elsewhere


In [30]:
# Run prediction on all images in the folder
results = model.predict(
    source='/content/drive/MyDrive/Military/military_object_dataset/test/images',  # folder path
    conf=0.25,
    save=True  # saves images with bounding boxes
)



WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1396 /content/drive/MyDrive/Military/military_object_dataset/test/images/000001.jpg: 640x640 (no detections), 8.0ms
image 2/1396 /content/drive/MyDrive/Military/military_object_dataset/test/images/000002.jpg: 640x640 (no detections), 8.1ms
image 3/1396 /content/drive/MyDrive/Military/military_object_dataset/test/images/000003.jpg: 640x640 (no detections), 8.4ms
image 4/1396 /content/drive/MyDrive/Military/military_object_dataset/test/images/000